# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
import os
import re
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
from wordcloud import WordCloud, STOPWORDS
import plotly.graph_objects as go


In [12]:
with open('../input/coleridgeinitiative-show-us-the-data/train/f8b03c87-9d1a-4f20-b76b-cb6c69d447b2.json') as F:
    sample = json.load(F)
    

In [13]:
sample[:1]

[{'section_title': 'Foreword',
  'text': 'The International Standard Classification of Education, known by its acronym ISCED, was developed by the United Nations Educational, Scientific, and Cultural Organization during the late 1960s and 1970s. ISCED was implemented in 1976 and is the recognized international standard for reporting and interpreting education program data. Creating a U.S. crosswalk to this system has been a goal of the National Center for Education Statistics and the Office of Research since the late 197,,s, when the National Institute of Education (the predecessor agency to the Office of Educational Research and Improvement) began exploring the idea. The design and implementation of a workable crosswalk, however, awaited the advent of changes to the Classification of Instructional Programs (CIP) system. The 1990 revision of the CIP system laid the foundation for a workable international crosswalk. Adoption of the National Education Goals set global consciousness and i

for n in sample:
    print(n['section_title'])

In [14]:
data_train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
data_train.sample(5)

,Id,pub_title,dataset_title,dataset_label,cleaned_label
17002,b25c93a7-f617-4fe6-916a-0ce8cb8bfa3d,Agricultural Contracting Update: Contracts in ...,Census of Agriculture,Census of Agriculture,census of agriculture
1068,99e64042-2a46-4a7a-890d-b9753d2e5fa1,"Circulation of Tampa Bay driven by buoyancy, t...",NOAA Tide Gauge,NOAA Tide Gauge,noaa tide gauge
1605,bc4b824a-c353-4be9-9dd2-2e48a500d43a,“Still Learning”: One Couple’s Literacy Develo...,Program for the International Assessment of Ad...,Program for the International Assessment of Ad...,program for the international assessment of ad...
2836,cb94ff7e-908f-47a4-bc7a-67b0d9b7b236,Examining social norm impacts on obesity and e...,Early Childhood Longitudinal Study,Early Childhood Longitudinal Study,early childhood longitudinal study
6210,20bdbb32-f8ac-4207-b6f8-2fc353a29034,Disrupted Control Network Connectivity in Abst...,Alzheimer's Disease Neuroimaging Initiative (A...,ADNI,adni


In [15]:
def get_text(filename, test=False):
    if test:
        df = pd.read_json('../input/coleridgeinitiative-show-us-the-data/test/{}.json'.format(filename))
    else:
        df = pd.read_json('../input/coleridgeinitiative-show-us-the-data/train/{}.json'.format(filename))
    text = " ".join(list(df['text']))
    return text

In [16]:
data_train['text'] = data_train['Id'].apply(get_text)
data_train.sample(5)

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text
15638,f64ac4a4-9171-48d5-a116-b8eb191ded42,Exploring the impact of dust on North Atlantic...,International Best Track Archive for Climate S...,International Best Track Archive for Climate S...,international best track archive for climate s...,High-resolution (Δx < 30 km) climate models ar...
430,5cc92e78-5ab3-4bda-85f2-5bfcee4af428,Impacts of 40 years of land cover change on wa...,Coastal Change Analysis Program,NOAA C-CAP,noaa c cap,Abstract: Land cover changes in the Tampa Bay ...
1313,58db06c6-c86d-4f84-898c-08fee74e2b02,Do Wealth Gains from Land Appreciation Cause F...,Agricultural Resource Management Survey,Agricultural Resource Management Survey,agricultural resource management survey,"In recent years, high crop prices and record f..."
1723,c3512f11-798f-4ae7-b486-bc9239e699e8,Sarcopenia and diabetes: Hyperglycemia is a ri...,Baltimore Longitudinal Study of Aging (BLSA),Baltimore Longitudinal Study of Aging,baltimore longitudinal study of aging,Skeletal muscle maintains our posture and prod...
1634,5582145f-7c65-4e1b-b443-ebb58801b287,Astro2020 APC White Paper: Findings and Recomm...,Survey of Earned Doctorates,Survey of Earned Doctorates,survey of earned doctorates,The Committee on the Status of Women in Astron...


In [19]:

data_train['cleaned_label']=data_train['cleaned_label'].astype("category")
data_train.dtypes

Id                 object
pub_title          object
dataset_title      object
dataset_label      object
cleaned_label    category
text               object
dtype: object

In [20]:


data_train['cleaned_label_cat']=data_train['cleaned_label'].cat.codes
data_train.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,cleaned_label_cat
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study used data from the National Educati...,65
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Dropping out of high school is not necessarily...,65
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,", stress satisfactory outcomes for all youth,...",65
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Federal Reserve Bank of Richmond S1. Accountin...,65
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor ...,65


In [ ]:
test_files = os.listdir('../input/coleridgeinitiative-show-us-the-data/test')
test_files

In [ ]:
test = pd.DataFrame({'Id':test_files})
test['Id'] = test['Id'].apply(lambda x : x.split('.')[0])
test['text'] = test['Id'].apply(get_text, test=True)
test

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data_train['text'])
X_train_counts.shape


(19661, 564514)

In [39]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(19661, 564514)

In [40]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, data_train['cleaned_label_cat'])

In [44]:
X_new_counts = count_vect.transform(test['text'])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
X_new_tfidf.shape

(4, 564514)

In [46]:
predicted = clf.predict(X_new_tfidf)

In [47]:
print(predicted)

[  3   3 126   3]


In [54]:
labels= data_train[['cleaned_label' , 'cleaned_label_cat']]
print(labels[labels['cleaned_label_cat']==3])
print(labels[labels['cleaned_label_cat']==126])

      cleaned_label  cleaned_label_cat
3419           adni                  3
3420           adni                  3
3421           adni                  3
3422           adni                  3
3423           adni                  3
...             ...                ...
16633          adni                  3
16684          adni                  3
16685          adni                  3
16686          adni                  3
16687          adni                  3

[3673 rows x 2 columns]
                                           cleaned_label  cleaned_label_cat
7086   trends in international mathematics and scienc...                126
7087   trends in international mathematics and scienc...                126
7088   trends in international mathematics and scienc...                126
7089   trends in international mathematics and scienc...                126
7090   trends in international mathematics and scienc...                126
...                                                

In [ ]:
words_in_titles = list(data_train.pub_title.str.split(expand=True).stack())

wordcloud = WordCloud(stopwords = STOPWORDS,
                      background_color = "white",
                      width = 1000,
                      height = 500
                     ).generate(' '.join(words_in_titles))
plt.figure(1, figsize = (16, 12))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
text = ' '.join(data_train['text'].sample(frac=0.3))
wordcloud = WordCloud(background_color='white', stopwords=STOPWORDS, width=2560, height=1440).generate(text)

barplot_dim = (15, 15)
ax = plt.subplots(figsize=barplot_dim, facecolor='w')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# A text cleaning function
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()